## Get Tweets in Real Time using Twitter API

- Use tweepy library to route all Live Tweets to a TCP Server. This server is hosted in cloud.
- OAuthHandler is used for authentication protocol(OAuth) of Twitter API.
- StreamListener listens to stream data we get from socket.

In [ ]:
import tweepy

from tweepy import OAuthHandler

from tweepy import Stream

from tweepy.streaming import StreamListener

- Socket is used to create socket connection with localhost.
- Json is used because twitter data comes in json format.

In [ ]:
import socket
import json

- To generate these keys create an app in https://apps.twitter.com.
- After creating the app, you'll get consumer_key & consumer_secret.
- Generate access_token & access_secret by using OAuth.

In [ ]:
consumer_key = 'HMNx5t9S6PJqglnb9kboYtZgq'
consumer_secret = 'kQMnxYlO1BFGjccvWiTLt69QdrYIEWE1X4iKz0MQqRnbrBHEPQ'
access_token = '3112767474-K2tNQiPx8MekqmqFZ4thh0OS5q4zkrAoJDY8NqC'
access_secret = '93oXUB570inlg8nO4foBTxgcuo2QLuGfJnbIIKwSfCdBl'

- class TweetListener is listening to stream via StreamListener & constructor initializes the socket.
- data contains the tweeets in json format coming from stream. In this json object field 'text' contains actual tweet.
- The actual tweet is extracted and sent to client socket. 

In [ ]:
class TweetsListener (StreamListener):
    def __init__ (self,csocket):
        self.client_socket = csocket
        
        def on_data(self, data):
            try:
                
                msg = json.loads(data)
                print(msg['text'].encoder('utf-8'))
                
                self.client_socket.send(msg['text'].encode('utf-8'))
                return True
        
            except BaseException as e:
                print("Error on_data: %s" % str(e))
                return True
        
        def on_error(self, status):
            print(status)
            return True
                

- auth is doing authentication check using OAuthHandler.
- twitter_stream will get the actual twitter live stream data. It'll call stream with auth & call the class TweetListener and     pass c_socket info to the class.
- from stream of tweets, filter tweets.

In [ ]:
def sendData(c_socket):
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    twitter_stream = Stream(auth, TwitterListener(c_socket))
    twitter_stream.filter(trace=['machinelearning'])

- Socket is created using socket function. Host is localhost where jupyter notebook is running.
- port is 9999 (It can be anything). Bind host and port address to the socket.

In [ ]:
s = socket.socket()
host = "127.0.0.1"
port = 9999
s.bind((host,port))
print("Listening on port: %s" %str(port))

- socket will wait and listen for few seconds after the connection is made.

In [ ]:
s.listen(5)
c, addr = s.accept()
print("Received request from:" + str(addr))

- sendData function will send all the filtered tweets to socket 'c' in real time.
- This will be consumed in next code of Spark Streaming to do analysis.

In [ ]:
sendData(c)